In [1]:
!pip install hume

     ---------------------------------------- 0.0/125.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/125.2 kB ? eta -:--:--
     --- ------------------------------------ 10.2/125.2 kB ? eta -:--:--
     --- ------------------------------------ 10.2/125.2 kB ? eta -:--:--
     ------------ ------------------------ 41.0/125.2 kB 245.8 kB/s eta 0:00:01
     -------------------------------- --- 112.6/125.2 kB 595.3 kB/s eta 0:00:01
     ------------------------------------ 125.2/125.2 kB 566.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/45.8 kB ? eta -:--:--
   ---------------------------------------- 45.8/45.8 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
   ---------------------------------------- 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/77.9 kB ? eta -:--:--
   ---------------------------------------- 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   -------------------

In [2]:
HUME_API_KEY = 'lQcg62KN7ErQGx7uK7EdRyv2cA8UZGTEpFQY2wfQJc22DxkO'

## Hume api ChatGPT

In [20]:
import os
from hume import HumeBatchClient
from hume.models.config import FaceConfig
import tempfile

# Replace with your Hume API key
api_key = os.getenv("HUME_API_KEY", HUME_API_KEY)

# Initialize Hume AI client
client = HumeBatchClient(api_key)

# Load image file
image_path = 'images/kevin_happy.jpg'
with open(image_path, 'rb') as image_file:
    image_bytes = image_file.read()

# Create a temporary file to store the image bytes
with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_image_file:
    temp_image_file.write(image_bytes)
    temp_image_file_path = temp_image_file.name

# Ensure the file is closed before it's passed to the API
try:
    # Configure the request for facial emotion recognition
    config = FaceConfig()

    # Send the request with the image file path
    job = client.submit_job(urls=None, files=[temp_image_file_path], configs=[config])

    # Wait for the result (this can take a few seconds)
    result = job.get_predictions()

    print(result)
    # Process the results
    # for face in result['faces']:
    #     emotions = face['emotions']
    #     for emotion in emotions:
    #         print(f"Emotion: {emotion['name']}, Score: {emotion['score']}")
finally:
    # Clean up the temporary file
    os.remove(temp_image_file_path)


AttributeError: 'HumeBatchClient' object has no attribute 'get_job_status'

## Hume AI Github Example

In [39]:
from hume import HumeBatchClient
from hume.models.config import FaceConfig, ProsodyConfig
import json
import os

# Initialize the Hume client with your API key
client = HumeBatchClient(HUME_API_KEY)

# Path to your local image file
image_path = 'images/kevin_happy.jpg'

# Submit a job with the local image file
job = client.submit_job(urls=None, files=[image_path], configs=[FaceConfig(), ProsodyConfig()])
job_id = job.id;

# Await job completion
job = job.await_complete()

# Get the job predictions
job_predictions = client.get_job_predictions(job_id=job_id)

# The start and end time range of predictions to be processed
start_time = 0
end_time = 12

# Top n emotions
n_top_values = 5

# A threshold of what is defined as a peaked emotion
peak_threshold = 0.7

emotions_dict = dict()
peaked_emotions_w_score_time_dict = dict()

# Process the predictions for facial expressions
for file in job_predictions:
    for prediction in file['results']['predictions']:
        for grouped_prediction in prediction['models']['face']['grouped_predictions']:
            for grouped_prediction_prediction in grouped_prediction['predictions']:
                if grouped_prediction_prediction['time'] >= start_time and grouped_prediction_prediction['time'] <= end_time:
                    for emotion in grouped_prediction_prediction['emotions']:
                        if emotion['name'] not in emotions_dict:
                            emotions_dict[emotion['name']] = emotion['score']
                        else:
                            emotions_dict[emotion['name']] += emotion['score']
                        if emotion['score'] >= peak_threshold:
                            peaked_emotions_w_score_time_dict[emotion['name']] = (emotion['score'], grouped_prediction_prediction['time'])

# Calculate average emotion scores
emotions_average = {emotion: score / len(emotions_dict) for emotion, score in emotions_dict.items()}

# Sort emotions by average score in descending order
ascend_sorted_emotion_average = sorted(emotions_average, key=emotions_average.get, reverse=True)

# Output the top emotions
print(f'The top {n_top_values} expressed emotions are between timestamp {start_time} and {end_time} : ')
for i in range(n_top_values):
    print(ascend_sorted_emotion_average[i])

# Output emotions that peaked over the threshold
print(f'The emotions that peaked over {peak_threshold} : ')
for peaked_emotions, (score, time) in peaked_emotions_w_score_time_dict.items():
    print(f"{peaked_emotions} with score of {score} at {time}")




The top 5 expressed emotions are between timestamp 0 and 12 : 
Joy
Amusement
Interest
Calmness
Excitement
The emotions that peaked over 0.7 : 


In [41]:
from hume import HumeBatchClient
from hume.models.config import FaceConfig, ProsodyConfig
import json
import os

# Initialize the Hume client with your API key
client = HumeBatchClient(HUME_API_KEY)

# Path to your local image file
def find_emotion(imgUrl):
    image_path = imgUrl
    
    # Submit a job with the local image file
    job = client.submit_job(urls=None, files=[image_path], configs=[FaceConfig(), ProsodyConfig()])
    job_id = job.id;
    
    # Await job completion
    job = job.await_complete()
    
    # Get the job predictions
    job_predictions = client.get_job_predictions(job_id=job_id)
    
    # The start and end time range of predictions to be processed
    start_time = 0
    end_time = 12
    
    # Top n emotions
    n_top_values = 5
    
    # A threshold of what is defined as a peaked emotion
    peak_threshold = 0.7
    
    emotions_dict = dict()
    peaked_emotions_w_score_time_dict = dict()
    
    # Process the predictions for facial expressions
    for file in job_predictions:
        for prediction in file['results']['predictions']:
            for grouped_prediction in prediction['models']['face']['grouped_predictions']:
                for grouped_prediction_prediction in grouped_prediction['predictions']:
                    if grouped_prediction_prediction['time'] >= start_time and grouped_prediction_prediction['time'] <= end_time:
                        for emotion in grouped_prediction_prediction['emotions']:
                            if emotion['name'] not in emotions_dict:
                                emotions_dict[emotion['name']] = emotion['score']
                            else:
                                emotions_dict[emotion['name']] += emotion['score']
                            if emotion['score'] >= peak_threshold:
                                peaked_emotions_w_score_time_dict[emotion['name']] = (emotion['score'], grouped_prediction_prediction['time'])
    
    # Calculate average emotion scores
    emotions_average = {emotion: score / len(emotions_dict) for emotion, score in emotions_dict.items()}
    
    # Sort emotions by average score in descending order
    ascend_sorted_emotion_average = sorted(emotions_average, key=emotions_average.get, reverse=True)
    
    # Output the top emotions
    print(f'The top {n_top_values} expressed emotions are between timestamp {start_time} and {end_time} : ')
    for i in range(n_top_values):
        print(ascend_sorted_emotion_average[i])
    
    # Output emotions that peaked over the threshold
    print(f'The emotions that peaked over {peak_threshold} : ')
    for peaked_emotions, (score, time) in peaked_emotions_w_score_time_dict.items():
        print(f"{peaked_emotions} with score of {score} at {time}")




In [43]:
find_emotion('images/monil2.jpg')

The top 5 expressed emotions are between timestamp 0 and 12 : 
Calmness
Boredom
Concentration
Confusion
Tiredness
The emotions that peaked over 0.7 : 
Boredom with score of 0.7193500399589539 at 0.0
Calmness with score of 0.7618798017501831 at 0.0


In [44]:
find_emotion('images/monil3.jpg')

The top 5 expressed emotions are between timestamp 0 and 12 : 
Confusion
Disappointment
Boredom
Anger
Surprise (negative)
The emotions that peaked over 0.7 : 
